In [1]:
import cv2
import mediapipe as mp
import json
import os

# Mediapipe kurulumu
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

# Video klasörü ve çıkış klasörü
video_folder = "İşaretDiliVideo"
output_folder = "json_verileri"

os.makedirs(output_folder, exist_ok=True)  # Çıkış klasörünü oluştur

# Videoları işle
for video_file in os.listdir(video_folder):
    if not video_file.endswith(".mp4"):
        continue  # Sadece MP4 dosyaları işle
    
    video_path = os.path.join(video_folder, video_file)
    cap = cv2.VideoCapture(video_path)
    holistic = mp_holistic.Holistic()

    keypoints_data = []  # Hareketleri saklayacağımız liste

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = holistic.process(frame_rgb)

        frame_data = {}

        if results.left_hand_landmarks:
            frame_data["left_hand"] = [[lm.x, lm.y, lm.z] for lm in results.left_hand_landmarks.landmark]
        
        if results.right_hand_landmarks:
            frame_data["right_hand"] = [[lm.x, lm.y, lm.z] for lm in results.right_hand_landmarks.landmark]

        keypoints_data.append(frame_data)

    cap.release()

    # JSON dosyasına kaydet
    json_path = os.path.join(output_folder, video_file.replace(".mp4", ".json"))
    with open(json_path, "w") as f:
        json.dump(keypoints_data, f)
    
    print(f"{video_file} işlendi ve {json_path} olarak kaydedildi.")

Ben.mp4 işlendi ve json_verileri\Ben.json olarak kaydedildi.
geliştiriyorum.mp4 işlendi ve json_verileri\geliştiriyorum.json olarak kaydedildi.
Merhaba.mp4 işlendi ve json_verileri\Merhaba.json olarak kaydedildi.
Proje.mp4 işlendi ve json_verileri\Proje.json olarak kaydedildi.
Öğrencisiyim.mp4 işlendi ve json_verileri\Öğrencisiyim.json olarak kaydedildi.
Üniversite.mp4 işlendi ve json_verileri\Üniversite.json olarak kaydedildi.
Şimdi.mp4 işlendi ve json_verileri\Şimdi.json olarak kaydedildi.


In [5]:
import open3d as o3d
import numpy as np
import time
import json
import os
from PIL import Image

json_folder = "json_verileri"
animation_output_folder = "animasyonlar"

os.makedirs(animation_output_folder, exist_ok=True)

# Tüm JSON dosyalarını oku
for json_file in os.listdir(json_folder):
    if not json_file.endswith(".json"):
        continue

    json_path = os.path.join(json_folder, json_file)
    
    with open(json_path, "r") as f:
        keypoints = json.load(f)

    # Open3D Robot Modeli
    torso = o3d.geometry.TriangleMesh.create_sphere(radius=0.15)  # Gövde
    shoulder = o3d.geometry.TriangleMesh.create_cylinder(radius=0.05, height=0.2)  # Omuz eklemi
    elbow = o3d.geometry.TriangleMesh.create_cylinder(radius=0.05, height=0.2)  # Dirsek eklemi
    wrist = o3d.geometry.TriangleMesh.create_cylinder(radius=0.05, height=0.2)  # Bilek eklemi
    hand = o3d.geometry.TriangleMesh.create_sphere(radius=0.08)  # El

    # Renkler
    torso.paint_uniform_color([0.2, 0.2, 0.8])  # Mavi gövde
    shoulder.paint_uniform_color([0.8, 0.2, 0.2])  # Kırmızı eklem
    elbow.paint_uniform_color([0.8, 0.2, 0.2])  # Kırmızı eklem
    wrist.paint_uniform_color([0.8, 0.2, 0.2])  # Kırmızı eklem
    hand.paint_uniform_color([0.2, 0.8, 0.2])  # Yeşil el

    # Başlangıç Pozisyonu
    shoulder.translate([0, 0, 0.2])
    elbow.translate([0, 0, 0.4])
    wrist.translate([0, 0, 0.6])
    hand.translate([0, 0, 0.8])

    # Görselleştirme Başlat
    vis = o3d.visualization.Visualizer()
    vis.create_window()

    vis.add_geometry(torso)
    vis.add_geometry(shoulder)
    vis.add_geometry(elbow)
    vis.add_geometry(wrist)
    vis.add_geometry(hand)

    frame_images = []  # GIF için kareleri saklama listesi

    # Hareketleri uygula
    for frame_idx, frame_data in enumerate(keypoints):
        if "right_hand" in frame_data and "right_elbow" in frame_data and "right_shoulder" in frame_data:
            # JSON'dan eklem koordinatlarını al
            shoulder_x, shoulder_y, shoulder_z = frame_data["right_shoulder"][0]
            elbow_x, elbow_y, elbow_z = frame_data["right_elbow"][0]
            wrist_x, wrist_y, wrist_z = frame_data["right_hand"][0]

            # Eklemleri yeni konumlara taşı
            shoulder.translate([shoulder_x, shoulder_y, shoulder_z], relative=False)
            elbow.translate([elbow_x, elbow_y, elbow_z], relative=False)
            wrist.translate([wrist_x, wrist_y, wrist_z], relative=False)
            hand.translate([wrist_x, wrist_y, wrist_z], relative=False)

            vis.update_geometry(shoulder)
            vis.update_geometry(elbow)
            vis.update_geometry(wrist)
            vis.update_geometry(hand)
            vis.poll_events()
            vis.update_renderer()
            time.sleep(0.05)

            # Kareyi kaydet
            frame_path = os.path.join(animation_output_folder, f"{json_file.replace('.json', '')}_{frame_idx}.png")
            vis.capture_screen_image(frame_path)
            frame_images.append(frame_path)

    vis.destroy_window()

    # 📌 Open3D Karelerini GIF'e Dönüştür
    gif_path = os.path.join(animation_output_folder, json_file.replace(".json", ".gif"))

    if frame_images:
        frames = [Image.open(frame) for frame in frame_images]
        frames[0].save(gif_path, save_all=True, append_images=frames[1:], duration=50, loop=0)

        # Geçici kareleri temizle
        for img in frame_images:
            os.remove(img)

        print(f"✅ Animasyon {gif_path} olarak kaydedildi.")
    else:
        print("⚠️ GIF oluşturulamadı, çünkü hiçbir kare kaydedilmedi.")

⚠️ GIF oluşturulamadı, çünkü hiçbir kare kaydedilmedi.
⚠️ GIF oluşturulamadı, çünkü hiçbir kare kaydedilmedi.
⚠️ GIF oluşturulamadı, çünkü hiçbir kare kaydedilmedi.
⚠️ GIF oluşturulamadı, çünkü hiçbir kare kaydedilmedi.
⚠️ GIF oluşturulamadı, çünkü hiçbir kare kaydedilmedi.
⚠️ GIF oluşturulamadı, çünkü hiçbir kare kaydedilmedi.
⚠️ GIF oluşturulamadı, çünkü hiçbir kare kaydedilmedi.


In [4]:
import speech_recognition as sr
import cv2
import os
import imageio
from PIL import Image, ImageSequence
import numpy as np

# 📂 Klasör Ayarları
gif_folder = "animasyonlar"  # GIF dosyalarının bulunduğu klasör
output_video = "output.mp4"  # Çıktı videosunun adı

# 🎙️ Konuşmayı Metne Dönüştür
def speech_to_text():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Konuşun...")
        recognizer.adjust_for_ambient_noise(source)
        audio = recognizer.listen(source)
    
    try:
        text = recognizer.recognize_google(audio, language="tr-TR")  # Türkçe destekli
        print(f"Algılanan Cümle: {text}")
        return text
    except sr.UnknownValueError:
        print("Ses anlaşılamadı.")
        return ""
    except sr.RequestError:
        print("Google API hatası.")
        return ""

# 📄 Metindeki Kelimeleri GIF Dosyalarıyla Eşleştir
def get_gif_files_for_words(text):
    words = text.lower().split()  # Cümledeki kelimeleri al
    gif_files = []

    for word in words:
        gif_path = os.path.join(gif_folder, f"{word}.gif")
        if os.path.exists(gif_path):
            gif_files.append(gif_path)
    
    return gif_files

# 📽️ GIF'leri Göster ve Kaydet
def display_gifs_and_save_video(gif_files, output_video):
    frame_size = (300, 300)  # Sabit boyut
    fps = 10  # Video FPS
    duration_per_gif = 2  # Her GIF'in süresi (saniye)

    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    out = cv2.VideoWriter(output_video, fourcc, fps, frame_size)

    for gif_path in gif_files:
        gif = Image.open(gif_path)
        frames = [frame.copy().resize(frame_size) for frame in ImageSequence.Iterator(gif)]
        
        for _ in range(duration_per_gif * fps):
            for frame in frames:
                frame_np = np.array(frame.convert("RGB"))
                frame_bgr = cv2.cvtColor(frame_np, cv2.COLOR_RGB2BGR)
                cv2.imshow("İşaret Dili", frame_bgr)
                out.write(frame_bgr)
                if cv2.waitKey(30) & 0xFF == ord("q"):
                    break
    
    out.release()
    cv2.destroyAllWindows()
    print(f"Video {output_video} olarak kaydedildi.")

# 🚀 Çalıştırma
text = speech_to_text()
if text:
    gif_files = get_gif_files_for_words(text)
    if gif_files:
        display_gifs_and_save_video(gif_files, output_video)
    else:
        print("Eşleşen GIF bulunamadı.")

ModuleNotFoundError: No module named 'imageio'